In [ ]:
import csv

birds_dataset = list()
with open('birds_dataset_alias.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count == 0:
            print(f'Column names are {", ".join(row)}')
            column_names = row
            line_count += 1
        else:
            birds_dataset.append(row)
            line_count += 1
    print(f'Processed {line_count} lines.')

birds_dataset_strings = list()
with open('birds_dataset.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count == 0:
            print(f'Column names are {", ".join(row)}')
            column_names = row
            line_count += 1
        else:
            birds_dataset_strings.append(row)
            line_count += 1
    print(f'Processed {line_count} lines.')

In [ ]:
from collections import Counter
import numpy as np

birds_dataset = np.array(birds_dataset, dtype='int32')
birds_dataset_strings = np.array(birds_dataset_strings)

In [ ]:
def get_scores(birds_dataset, normalize=True):
    nfts_number = len(birds_dataset)
    columns_number = len(birds_dataset[0])
    birds_scores = np.zeros((nfts_number,))
    for column_index in range(columns_number):
        bird_traits = birds_dataset[:, column_index]
        entries = Counter(bird_traits)
        bird_group_sizes = np.array([entries[x] for x in bird_traits]).reshape((nfts_number, 1))
        keys = entries.keys()
        values = np.array(list(entries.values()))
        groups_number = len(values)
        values = values.reshape((1, len(values)))
        current_trait_score = (np.sum(np.divide((values + np.zeros((nfts_number, 1))), (bird_group_sizes + values)), 1) - 1/2) / (groups_number - 1)
        if normalize:
            current_trait_score = current_trait_score / np.sum(current_trait_score) * nfts_number
        birds_scores = birds_scores + current_trait_score  
    return birds_scores / columns_number

In [ ]:
birds_scores = get_scores(birds_dataset[:, 1:])
birds_trait_scores = birds_scores

In [ ]:
def get_set_scores(birds_dataset, normalize=True):
    nfts_number = len(birds_dataset)
    columns_number = len(birds_dataset[0])
    # get biggest subset
    bigest_subsets = list()
    for bird_index in range(len(birds_dataset)):
            current_list = list(birds_dataset_strings[bird_index, (1, 2, 3, 4, 5, 6, 7, 8, 9)])
            entries = Counter(current_list)
            current_set_strings = list()
            current_set_sizes = list()
            for set_string in list(entries.keys()):
                if (set_string.find('var') == -1):
                    current_set_strings.append(set_string)
                    string_enrty_counts = 0
                    for trait_index in [2, 4, 5, 6, 7, 8]:
                        if birds_dataset_strings[bird_index, trait_index] == set_string:
                            string_enrty_counts = string_enrty_counts + 1
                    current_set_sizes.append(string_enrty_counts)
            bigest_subsets.append(sorted(current_set_sizes, key=lambda x:-x)[0])
    bigest_subset_sizes = list()
    for subset_size in range(7):
        bigest_subset_sizes.append(np.sum([x == subset_size for x in bigest_subsets]))
    bigest_subset_sizes = np.array(bigest_subset_sizes)

    # get enhancement subset and its cardinality
    set_enhancement_sizes = np.zeros((nfts_number,))
    set_enhancement_cardinality = np.zeros((nfts_number,))
    for subset_size in range(7):
        indexes = list()
        for bird_index in range(len(birds_dataset)):
            if bigest_subsets[bird_index] == subset_size:
                indexes.append(bird_index)
        for bird_index in indexes:
            current_list = list(birds_dataset_strings[bird_index, (1, 2, 3, 4, 5, 6, 7, 8, 9)])
            entries = Counter(current_list)
            current_set_strings = list()
            current_set_sizes = list()
            current_set_sizes.append(0)
            for set_string in list(entries.keys()):
                if (set_string.find('var') == -1):
                    current_set_strings.append(set_string)
                    string_enrty_counts = 0
                    for trait_index in [2, 4, 5, 6, 7, 8]:
                        if birds_dataset_strings[bird_index, trait_index] == set_string:
                            string_enrty_counts = string_enrty_counts + 1       
                    if string_enrty_counts == bigest_subsets[bird_index]:
                        string_enrty_counts = 0
                        for trait_index in [3, 9]:
                            if birds_dataset_strings[bird_index, trait_index] == set_string:
                                string_enrty_counts = string_enrty_counts + 1
                        for trait_index in [1]:
                            if birds_dataset_strings[bird_index, trait_index] == set_string:
                                string_enrty_counts = string_enrty_counts + 10
                                # print(bigest_subsets[bird_index], birds_dataset_strings[bird_index, 0], birds_dataset_strings[bird_index, 1])
                        current_set_sizes.append(string_enrty_counts)
            set_enhancement_sizes[bird_index] = max(current_set_sizes)
        entries = Counter(set_enhancement_sizes[indexes])
        for bird_index in indexes:
            set_enhancement_cardinality[bird_index] = entries[set_enhancement_sizes[bird_index]]
    # compute scores (make matrices to speed up)
    birds_scores = np.zeros((nfts_number,))    
    for bird_index_1 in range(len(birds_dataset)):
        for bird_index_2 in range(bird_index_1):
            if bigest_subsets[bird_index_1] > bigest_subsets[bird_index_2]:
                birds_scores[bird_index_1] = birds_scores[bird_index_1] + 1
            elif bigest_subsets[bird_index_1] < bigest_subsets[bird_index_2]:
                birds_scores[bird_index_2] = birds_scores[bird_index_2] + 1
            else: # ==
                birds_scores[bird_index_1] = birds_scores[bird_index_1] + set_enhancement_cardinality[bird_index_2]/(
                    set_enhancement_cardinality[bird_index_1] + set_enhancement_cardinality[bird_index_2])
                birds_scores[bird_index_2] = birds_scores[bird_index_2] + set_enhancement_cardinality[bird_index_1]/(
                    set_enhancement_cardinality[bird_index_1] + set_enhancement_cardinality[bird_index_2])
    birds_scores = birds_scores / (nfts_number - 1)
    if normalize:
        birds_scores = birds_scores / np.sum(birds_scores) * nfts_number
    return birds_scores

In [ ]:
birds_set_scores = get_set_scores(birds_dataset)
print(np.mean(birds_set_scores))

In [ ]:
def get_edition_scores(birds_dataset, normalize=True):
    nfts_number = len(birds_dataset)
    borders = [10, 100, 1000]
    
    birds_scores = np.zeros((nfts_number,))
    edition_trait = np.zeros((nfts_number,))
    for current_border in borders:
        edition_trait = edition_trait + (birds_dataset[:, 0] < current_border)
    entries = Counter(edition_trait)
    bird_group_sizes = np.array([entries[x] for x in edition_trait]).reshape((nfts_number, 1))
    keys = entries.keys()
    values = np.array(list(entries.values()))
    groups_number = len(values)
    values = values.reshape((1, len(values)))
    birds_scores = birds_scores + (np.sum(np.divide((values + np.zeros((nfts_number, 1))), (bird_group_sizes + values)), 1) - 1/2) / (groups_number - 1)
    if normalize:
        birds_scores = birds_scores / np.sum(birds_scores) * nfts_number
    return birds_scores

In [ ]:
birds_edition_scores = get_edition_scores(birds_dataset)
np.shape(birds_edition_scores)

In [ ]:
import scipy.stats as ss

nfts_number = len(birds_dataset)

birds_trait_ranks = (nfts_number + 1) - ss.rankdata(birds_trait_scores, method='max')
birds_trait_ranks = [int(x) for x in birds_trait_ranks]

birds_set_ranks = (nfts_number + 1) - ss.rankdata(birds_set_scores, method='max')
birds_set_ranks = [int(x) for x in birds_set_ranks]

birds_edition_ranks = (nfts_number + 1) - ss.rankdata(birds_edition_scores, method='max')
birds_edition_ranks = [int(x) for x in birds_edition_ranks]

In [ ]:
birds_weighted_scores = (birds_trait_scores + birds_set_scores + birds_edition_scores) / 3
birds_weighted_ranks = (nfts_number + 1) - ss.rankdata(birds_weighted_scores, method='max')
birds_weighted_ranks = [int(x) for x in birds_weighted_ranks]

In [ ]:
birds_dataset_with_ranks = np.hstack((birds_dataset_strings, 
                                      np.reshape(birds_trait_scores, (nfts_number, 1)),
                                      np.reshape(birds_trait_ranks, (nfts_number, 1)),
                                      np.reshape(birds_set_scores, (nfts_number, 1)),
                                      np.reshape(birds_set_ranks, (nfts_number, 1)),
                                      np.reshape(birds_edition_scores, (nfts_number, 1)),
                                      np.reshape(birds_edition_ranks, (nfts_number, 1)),
                                      np.reshape(birds_weighted_scores, (nfts_number, 1)),
                                      np.reshape(birds_weighted_ranks, (nfts_number, 1))
                                     ))

In [ ]:
column_names = ['bird_id', 'theme', 'head', 'eyes', 'body', 'tail', 'wingLeft', 'wingRight', 'feet', 'beak',
                'trait_score', 'trait_rank', 'set_score', 'set_rank', 'edition_score', 'edition_rank', 
                'weighted_score', 'weighted_rank']

with open('birds_dataset_with_ranks.csv', 'w', newline='') as birds_file:
    bird_writer = csv.writer(birds_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    bird_writer.writerow(column_names)
    for row_index in range(len(birds_dataset_with_ranks)):
        bird_writer.writerow(birds_dataset_with_ranks[row_index])

birds_dataset_with_ranks = list()
int_columns = [0, -1, -3, -5, -7]
numeric_columns = [-2, -4, -6, -8]
with open('birds_dataset_with_ranks.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count == 0:
            print(f'Column names are {", ".join(row)}')
            column_names = row
            line_count += 1
        else:
            for int_element in int_columns:
                row[int_element] = int(row[int_element])
            for numeric_element in numeric_columns:
                row[numeric_element] = float(row[numeric_element])
            birds_dataset_with_ranks.append(row)
            line_count += 1
    print(f'Processed {line_count} lines.')

# Visualize ranks

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

bird_indexes = np.array([x[0] for x in birds_dataset_with_ranks])
bird_ranks = np.array([x[-1] for x in birds_dataset_with_ranks])
borders = [0, 10, 100, 1000, 10000]

birds_scores = np.zeros((nfts_number,))
edition_trait = np.zeros((nfts_number,))
for current_border in borders:
    edition_trait = edition_trait + (bird_indexes < current_border)
edition_trait = [int(x) for x in edition_trait]
edition_collections_number = len(np.unique(edition_trait))
data = np.zeros((edition_collections_number, edition_collections_number))
for row in range(edition_collections_number):
    for column in range(edition_collections_number):
        for bird_index in range(len(bird_indexes)):
            if ((bird_indexes[bird_index] >= borders[row]) and 
                (bird_indexes[bird_index] < borders[row + 1]) and
                (bird_ranks[bird_index] >= borders[column]) and
                (bird_ranks[bird_index] < borders[column + 1])
               ):
                data[row, column] = data[row, column] + 1
columns = ('Among 1-9', '10-99', '100-999', '1000-9999')
rows = ['Super Founder', 'Founder', 'Rare', 'Limited Edition']

# Get some pastel shades for the colors
colors = ['orange', 'blueviolet', 'blue', 'limegreen']# plt.cm.BuPu(np.linspace(0, 0.5, len(rows)))
n_rows = len(data)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

fig, ax = plt.subplots()

# hide axes
fig.patch.set_visible(False)
ax.axis('off')
ax.axis('tight')

df = pd.DataFrame(data, columns=columns, index=rows).astype(int)

ax.table(cellText=df.values, rowLabels=rows, rowColours=colors, colLabels=df.columns, colColours=colors, loc='center')

fig.tight_layout()

plt.show()

In [ ]:
indexes = [-8, -6, -4, -2]
for index in indexes:
    values = list()
    for bird_index in range(len(birds_dataset_with_ranks)):
        values.append(birds_dataset_with_ranks[bird_index][index])
    print(np.mean(values))

In [ ]:
indexes = [-8, -6, -4, -2]
for index in indexes:
    values = list()
    for bird_index in range(len(birds_dataset_with_ranks)):
        values.append(birds_dataset_with_ranks[bird_index][index])
    print(np.mean(np.unique(values)))

In [ ]:
print(birds_dataset_with_ranks[0])